In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:08<00:00, 1142.13it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:376: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("bin")


{'basic': {'expected_sample_burst_size': 2.9978442855603062,
           'expected_sample_data_size': 16.486670499047893,
           'expected_time_to_recover': 99.1673,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16134350738600325,
             'bit_redundancy': 415.8365,
             'ideal_bit_efficency': 0.19068131162351254,
             'ideal_bit_redundancy': 339.5481944132765,
             'ideal_packet_efficiency': 0.9534065581175627,
             'ideal_packet_redundancy': 67.9096388826553,
             'ideal_time_to_recover': 83.9096388826553,
             'ideal_transmitted_bits': 419.5481944132765,
             'packet_efficiency': 0.8067175369300162,
             'packet_redundancy': 83.1673,
             'permeability': 0.8461422150512851,
             'saturation': 16.0,
             'transmitted_bits': 495.8365}}
Found implementation: anfext2
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:05<00:00, 1873.59it/s]


{'basic': {'expected_sample_burst_size': 2.9949595967677416,
           'expected_sample_data_size': 16.56764541163293,
           'expected_time_to_recover': 39.55919999999999,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.40445711743412416,
             'bit_redundancy': 117.79599999999994,
             'ideal_bit_efficency': 0.4775714734722981,
             'ideal_bit_redundancy': 87.5141930449505,
             'ideal_packet_efficiency': 2.3878573673614905,
             'ideal_packet_redundancy': 17.502838608990103,
             'ideal_time_to_recover': 33.5028386089901,
             'ideal_transmitted_bits': 167.5141930449505,
             'packet_efficiency': 2.0222855871706207,
             'packet_redundancy': 23.55919999999999,
             'permeability': 0.8469038456033012,
             'saturation': 16.0,
             'transmitted_bits': 197.79599999999994}}
Found implementation: anfext2l
  First time seen. Benchmarking.


...

{'basic': {'expected_sample_burst_size': 2.984223245634034,
           'expected_sample_data_size': 16.446126487620262,
           'expected_time_to_recover': 32.4093,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.15427670452616996,
             'bit_redundancy': 137.0465,
             'ideal_bit_efficency': 0.1822709029323035,
             'ideal_bit_redundancy': 112.1584800306012,
             'ideal_packet_efficiency': 0.9113545146615175,
             'ideal_packet_redundancy': 22.43169600612024,
             'ideal_time_to_recover': 27.43169600612024,
             'ideal_transmitted_bits': 137.1584800306012,
             'packet_efficiency': 0.7713835226308497,
             'packet_redundancy': 27.4093,
             'permeability': 0.8464143318775857,
             'saturation': 5.0,
             'transmitted_bits': 162.0465}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 2876.09it/s]


{'basic': {'expected_sample_burst_size': 3.0062808077595804,
           'expected_sample_data_size': 16.506837335029417,
           'expected_time_to_recover': 13.198699999999999,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.35235901003544523,
             'bit_redundancy': 42.740095671225845,
             'ideal_bit_efficency': 0.41653181963009656,
             'ideal_bit_redundancy': 32.5728332667701,
             'ideal_packet_efficiency': 2.082659098150483,
             'ideal_packet_redundancy': 6.514566653354018,
             'ideal_time_to_recover': 11.16524751910885,
             'ideal_transmitted_bits': 55.82623759554425,
             'packet_efficiency': 1.7617950501772262,
             'packet_redundancy': 8.548019134245168,
             'permeability': 0.8459353966003357,
             'saturation': 4.650680865754831,
             'transmitted_bits': 65.9935}}
Found implementation: single_binary_fullrank2
  Fir

benchmark: 100%|██████████| 10000/10000 [00:03<00:00, 2778.71it/s]


{'basic': {'expected_sample_burst_size': 3.006423982869379,
           'expected_sample_data_size': 16.594020144341343,
           'expected_time_to_recover': 13.814000000000004,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3366643163279882,
             'bit_redundancy': 45.81659567122587,
             'ideal_bit_efficency': 0.3976595221901395,
             'ideal_bit_redundancy': 35.22225897410438,
             'ideal_packet_efficiency': 1.9882976109506978,
             'ideal_packet_redundancy': 7.0444517948208745,
             'ideal_time_to_recover': 11.695132660575705,
             'ideal_transmitted_bits': 58.47566330287853,
             'packet_efficiency': 1.6833215816399412,
             'packet_redundancy': 9.163319134245173,
             'permeability': 0.8466144969288911,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.07000000000002}}


In [3]:
# TODO: more datamining - plots of progression of ttr distributions wrt payload, erasure etc, comparing approaches in this way